First exploration of a new strategy: Inside Bar Momentum.  Simply:  if high-low of bar 2 within high-low of bar 1: signal...

In [1]:
import pandas as pd
import plotly.graph_objects as go
import utils

We know we are going to look for data on USD_JPY_H4, but we haven't yet tested collect_his_data with this pairing, so we need to run that first.  Might as well refresh while we're in the area...

But, this takes a long time to run - so comment out after doing it once.

In [2]:
# import os
# %run ./collect_his_data.py


In [3]:
def plot_candles_mode(df_plot, mode=None):

    if mode == None:
        mode = "markers"

    plot_colours_buy = ['#043ef9', '#eb5334', '#34eb37']
    plot_colours_sell = ['white', 'red', 'yellow']

    fig = go.Figure()
    
    fig.add_trace(go.Candlestick(
        x=df_plot.time, open=df_plot.mid_o, high=df_plot.mid_h, low = df_plot.mid_l, close=df_plot.mid_c,
        line=dict(width=1), opacity=1,
        increasing_fillcolor="#24A06B",
        decreasing_fillcolor="#CC2E3C",
        increasing_line_color="#2EC886",
        decreasing_line_color="#FF3A4C"
        ))

    # Loop through buys, and plot
    for i in range(0, 3):
        fig.add_trace(go.Scatter(
            x=df_buys.time,
            y=df_buys[plot_cols[i]],
            mode=mode,
            name=(f"Buy {plot_cols[i]}"),
            marker=dict(color=plot_colours_buy[i], size=12),
            
        ))

    # Loop through sells and plot
    for i in range(0, 3):
        fig.add_trace(go.Scatter(
            x=df_sells.time,
            y=df_sells[plot_cols[i]],
            mode=mode,
            name=(f"Sell {plot_cols[i]}"),
            marker=dict(color=plot_colours_sell[i], size=12)
        ))


    fig.update_layout(width=1000, height=400, paper_bgcolor = "#1e1e1e", plot_bgcolor = "#1e1e1e",
                    margin=dict(l=10, b=10, t=30, r=10), 
                    font=dict(size=10, color="#e1e1e1"),
                    title=f"Buy/Sell {mode} Chart")
    fig.update_xaxes(gridcolor="#1f292f",
                    showgrid=True,
                    fixedrange=True,
                    rangeslider=dict(visible=False),
                    rangebreaks=[
                        dict(bounds=["sat", "mon"])
                        ]
                    )
    fig.update_yaxes(gridcolor="#1f292f",
                    showgrid=True)

    fig.show()

In [4]:
pair = "USD_JPY"
granularity = "H4"
df_raw = pd.read_csv(utils.get_hist_data_filename(pair, granularity))


In [5]:
df_raw.describe()

,Unnamed: 0,volume,bid_o,bid_h,bid_l,bid_c,mid_o,mid_h,mid_l,mid_c,ask_o,ask_h,ask_l,ask_c
count,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000
mean,667.013904,17061.963422,116.006735,116.177761,115.836413,116.015489,116.019676,116.186209,115.847156,116.025663,116.032627,116.196451,115.855979,116.035833
std,419.124282,13381.055089,12.734903,12.797093,12.658796,12.737169,12.735869,12.797949,12.659539,12.737199,12.736837,12.798409,12.660817,12.737247
min,0.000000,8.000000,102.064000,102.556000,101.170000,102.063000,102.072000,102.566000,101.180000,102.070000,102.079000,102.579000,101.190000,102.077000
25%,292.000000,7905.500000,107.191000,107.317000,107.072000,107.195000,107.202500,107.325000,107.086000,107.206000,107.215000,107.334000,107.094000,107.217500
50%,643.000000,13316.000000,109.985000,110.099000,109.884000,109.995000,110.002000,110.107000,109.892000,110.001000,110.010000,110.117000,109.898000,110.007000
75%,1033.000000,21582.000000,122.641500,122.949000,122.472000,122.697000,122.648000,122.957000,122.482000,122.705000,122.665500,122.966000,122.492000,122.712500
max,1424.000000,114647.000000,151.467000,151.938000,150.767000,151.465000,151.480000,151.946000,150.780000,151.478000,151.494000,151.956000,150.793000,151.492000


In [6]:
non_nums = ['ticker', 'time', 'volume']
num_cols = [x for x in df_raw.columns if x not in non_nums]
df_raw[num_cols] = df_raw[num_cols].apply(pd.to_numeric)



In [7]:
df_raw.columns

Index(['Unnamed: 0', 'ticker', 'time', 'volume', 'bid_o', 'bid_h', 'bid_l',
       'bid_c', 'mid_o', 'mid_h', 'mid_l', 'mid_c', 'ask_o', 'ask_h', 'ask_l',
       'ask_c'],
      dtype='object')

In [8]:
df = df_raw[['ticker', 'time', 'mid_o', 'mid_h', 'mid_l', 'mid_c', 'ask_c']].copy()

In [9]:
def direction(row):
    """ Return price direction based on close vs open price i.e. if close higher, upwards (1)..."""
    if row.mid_c > row.mid_o:
        return 1
    return -1

In [10]:
def direction(row):
    """ Return price direction based on close vs open price i.e. if close higher, upwards (1)..."""
    if row.mid_c > row.mid_o:
        return 1
    return -1

In [11]:
SLOSS = 0.4
TPROFIT = 0.8
ENTRY_PRC = 0.1

def direction(row):
    """ Return price direction based on close vs open price i.e. if close higher, upwards (1)..."""
    if row.mid_c > row.mid_o:
        return 1
    return -1

def get_signal(row):
    """  Identify encapsulation in previous candle and return direction.  Pass back zero if no encapsulation.  
    1 = Buy, -1=Sell:  Buy if encapsulated in a previously upward candle...      """
    if row.mid_h_prev > row.mid_h and row.mid_l_prev > row.mid_l:
        return row.DIRECTION_prev
    return 0

def get_entry_stop(row):
    if row.SIGNAL == 1:
        return (row.RANGE_prev * ENTRY_PRC) + row.ask_h_prev
    elif row.SIGNAL == -1:
        return row.bid_l_prev - (row.RANGE_prev * ENTRY_PRC)
    else:
        return 0
    
def get_stop_loss(row):
    if row.SIGNAL == 1:
        return row.ENTRY - (row.RANGE_prev * SLOSS)
    if row.SIGNAL == -1:
        return row.ENTRY + (row.RANGE_prev * SLOSS)
    else:
        return 0
    
def get_take_profit(row):
    if row.SIGNAL == 1:
        return row.ENTRY + (row.RANGE_prev * TPROFIT)
    if row.SIGNAL == -1:
        return row.ENTRY - (row.RANGE_prev * TPROFIT)
    else:
        return 0

## Additional column capture/derivation.

* The range of a candle is the difference between the high and low price.
* To track whether the high and low and enclosed in previous candle, we need high and low values from previous.



In [12]:
#TODO:  Create a new dataframe/sheet to capture Data Dictionary and some metadata...

df = df_raw.copy()
df['RANGE'] = df.mid_h - df.mid_l
df['mid_h_prev'] = df.mid_h.shift(1)
df['mid_l_prev'] = df.mid_l.shift(1)
df['ask_h_prev'] = df.ask_h.shift(1)
df['bid_l_prev'] = df.bid_l.shift(1)
df['RANGE_prev'] = df.RANGE.shift(1)
df['DIRECTION'] = df.apply(direction, axis=1)
df['DIRECTION_prev'] = df.DIRECTION.shift(1).fillna(0).astype(int)
df.dropna(inplace=True)
df['SIGNAL'] = df.apply(get_signal, axis=1)
df.reset_index(drop=True, inplace=True)
df.head()


,Unnamed: 0,ticker,time,volume,bid_o,bid_h,bid_l,bid_c,mid_o,mid_h,...,ask_c,RANGE,mid_h_prev,mid_l_prev,ask_h_prev,bid_l_prev,RANGE_prev,DIRECTION,DIRECTION_prev,SIGNAL
0,1,USD_JPY,2020-01-02T02:00:00.000000000Z,579,108.664,108.751,108.652,108.736,108.671,108.758,...,108.747,0.100,108.761,108.607,108.808,108.586,0.154,1,1,0
1,2,USD_JPY,2020-01-02T06:00:00.000000000Z,1418,108.738,108.836,108.688,108.833,108.744,108.842,...,108.845,0.146,108.758,108.658,108.764,108.652,0.100,1,1,0
2,3,USD_JPY,2020-01-02T10:00:00.000000000Z,1134,108.830,108.859,108.652,108.688,108.836,108.866,...,108.700,0.208,108.842,108.696,108.848,108.688,0.146,-1,1,0
3,4,USD_JPY,2020-01-02T14:00:00.000000000Z,3273,108.691,108.742,108.206,108.523,108.697,108.748,...,108.535,0.536,108.866,108.658,108.872,108.652,0.208,-1,-1,-1
4,5,USD_JPY,2020-01-02T18:00:00.000000000Z,1029,108.520,108.566,108.488,108.511,108.526,108.572,...,108.611,0.078,108.748,108.212,108.754,108.206,0.536,1,-1,0


Capture how many rows in our dataframe, and count how many buy/sell signals we're getting.

In [13]:
print(f"total number of rows:  {df.shape[0]}")
df.groupby(by="SIGNAL").count()

total number of rows:  4674


,Unnamed: 0,ticker,time,volume,bid_o,bid_h,bid_l,bid_c,mid_o,mid_h,...,ask_l,ask_c,RANGE,mid_h_prev,mid_l_prev,ask_h_prev,bid_l_prev,RANGE_prev,DIRECTION,DIRECTION_prev
SIGNAL,,,,,,,,,,,,,,,,,,,,,
-1,1112,1112,1112,1112,1112,1112,1112,1112,1112,1112,...,1112,1112,1112,1112,1112,1112,1112,1112,1112,1112
0,3161,3161,3161,3161,3161,3161,3161,3161,3161,3161,...,3161,3161,3161,3161,3161,3161,3161,3161,3161,3161
1,401,401,401,401,401,401,401,401,401,401,...,401,401,401,401,401,401,401,401,401,401


In [14]:
df['ENTRY'] = df.apply(get_entry_stop, axis=1)
df['STOPLOSS'] = df.apply(get_stop_loss, axis=1)
df['TAKEPROFIT'] = df.apply(get_take_profit, axis=1)

df_plot = df.iloc[0:60]
df_buys = df_plot[df_plot.SIGNAL == 1]
df_sells = df_plot[df_plot.SIGNAL == -1]

plot_cols = ['ENTRY', 'STOPLOSS', 'TAKEPROFIT']





In [15]:
df['ENTRY'] = df.apply(get_entry_stop, axis=1)
df['STOPLOSS'] = df.apply(get_stop_loss, axis=1)
df['TAKEPROFIT'] = df.apply(get_take_profit, axis=1)
df[df.SIGNAL==1].head()


,Unnamed: 0,ticker,time,volume,bid_o,bid_h,bid_l,bid_c,mid_o,mid_h,...,mid_l_prev,ask_h_prev,bid_l_prev,RANGE_prev,DIRECTION,DIRECTION_prev,SIGNAL,ENTRY,STOPLOSS,TAKEPROFIT
11,12,USD_JPY,2020-01-05T22:00:00.000000000Z,4564,107.854,108.115,107.764,107.941,107.879,108.122,...,107.842,108.160,107.835,0.284,1,1,1,108.1884,108.0748,108.4156
17,18,USD_JPY,2020-01-06T22:00:00.000000000Z,1722,108.303,108.455,108.280,108.429,108.353,108.461,...,108.342,108.513,108.335,0.164,1,1,1,108.5294,108.4638,108.6606
19,20,USD_JPY,2020-01-07T06:00:00.000000000Z,3475,108.432,108.494,108.255,108.462,108.438,108.500,...,108.423,108.513,108.417,0.083,1,1,1,108.5213,108.4881,108.5877
34,35,USD_JPY,2020-01-09T18:00:00.000000000Z,1707,109.503,109.532,109.385,109.515,109.510,109.538,...,109.429,109.588,109.422,0.151,1,1,1,109.6031,109.5427,109.7239
62,63,USD_JPY,2020-01-16T10:00:00.000000000Z,6412,109.997,110.049,109.918,110.007,110.002,110.054,...,109.932,110.075,109.926,0.134,1,1,1,110.0884,110.0348,110.1956


In [16]:
df_buys.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 4 entries, 11 to 34
Data columns (total 28 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      4 non-null      int64  
 1   ticker          4 non-null      object 
 2   time            4 non-null      object 
 3   volume          4 non-null      int64  
 4   bid_o           4 non-null      float64
 5   bid_h           4 non-null      float64
 6   bid_l           4 non-null      float64
 7   bid_c           4 non-null      float64
 8   mid_o           4 non-null      float64
 9   mid_h           4 non-null      float64
 10  mid_l           4 non-null      float64
 11  mid_c           4 non-null      float64
 12  ask_o           4 non-null      float64
 13  ask_h           4 non-null      float64
 14  ask_l           4 non-null      float64
 15  ask_c           4 non-null      float64
 16  RANGE           4 non-null      float64
 17  mid_h_prev      4 non-null      float

In [17]:
class Trade():
    def __init__(self, row):
        self.candle_date = row.time
        self.direction = row.SIGNAL
        self.entry = row.ENTRY
        self.TP = row.TAKEPROFIT
        self.SL = row.STOPLOSS
        self.running = False
        self.result = None
        self.stopped = None

    def update(self, row):
        if self.running == True:
            self.update_result(row)
        else:
            self.check_entry(row)

    def check_entry(self, row):
        if self.direction == 1 and row.mid_c >= self.entry or self.direction == -1 and row.mid_c <= self.entry:
            self.index = row.name
            self.opened = row.time
            self.running = True

    def update_result(self, row):
        if self.direction == 1:
            if row.mid_c >= self.TP:
                self.result = 2.0
            elif row.mid_c <= self.SL:
                self.result = -1.0
        else:
            if row.mid_c <= self.TP:
                self.result = 2.0
            elif row.mid_c >= self.SL:
                self.result = -1.0

        if self.result is not None:
            self.running = False
            self.stopped = row.time



In [18]:
open_trades = []
closed_trades = []

for index, row in df.iterrows():
    for ot in open_trades:
        ot.update(row)
        if ot.stopped is not None:
            closed_trades.append(ot)

    open_trades = [x for x in open_trades if x.stopped is None]

    if row.SIGNAL != 0:
        open_trades = [x for x in open_trades if x.running == True]
        open_trades.append(Trade(row))


    

In [19]:
len(closed_trades)

780

In [20]:
df_trades = pd.DataFrame.from_dict([vars(x) for x in closed_trades])
df_trades.head()

,candle_date,direction,entry,TP,SL,running,result,stopped,index,opened
0,2020-01-02T14:00:00.000000000Z,-1,108.6312,108.4648,108.7144,False,2.0,2020-01-02T22:00:00.000000000Z,4,2020-01-02T18:00:00.000000000Z
1,2020-01-03T06:00:00.000000000Z,-1,107.9698,107.7762,108.0666,False,-1.0,2020-01-03T18:00:00.000000000Z,9,2020-01-03T14:00:00.000000000Z
2,2020-01-03T02:00:00.000000000Z,-1,108.1138,107.7442,108.2986,False,-1.0,2020-01-06T14:00:00.000000000Z,7,2020-01-03T06:00:00.000000000Z
3,2020-01-05T22:00:00.000000000Z,1,108.1884,108.4156,108.0748,False,2.0,2020-01-06T22:00:00.000000000Z,15,2020-01-06T14:00:00.000000000Z
4,2020-01-07T06:00:00.000000000Z,1,108.5213,108.5877,108.4881,False,-1.0,2020-01-07T18:00:00.000000000Z,21,2020-01-07T14:00:00.000000000Z


In [21]:
df_trades.result.sum()

744.0

In [22]:
df[df.SIGNAL==1].head()

,Unnamed: 0,ticker,time,volume,bid_o,bid_h,bid_l,bid_c,mid_o,mid_h,...,mid_l_prev,ask_h_prev,bid_l_prev,RANGE_prev,DIRECTION,DIRECTION_prev,SIGNAL,ENTRY,STOPLOSS,TAKEPROFIT
11,12,USD_JPY,2020-01-05T22:00:00.000000000Z,4564,107.854,108.115,107.764,107.941,107.879,108.122,...,107.842,108.160,107.835,0.284,1,1,1,108.1884,108.0748,108.4156
17,18,USD_JPY,2020-01-06T22:00:00.000000000Z,1722,108.303,108.455,108.280,108.429,108.353,108.461,...,108.342,108.513,108.335,0.164,1,1,1,108.5294,108.4638,108.6606
19,20,USD_JPY,2020-01-07T06:00:00.000000000Z,3475,108.432,108.494,108.255,108.462,108.438,108.500,...,108.423,108.513,108.417,0.083,1,1,1,108.5213,108.4881,108.5877
34,35,USD_JPY,2020-01-09T18:00:00.000000000Z,1707,109.503,109.532,109.385,109.515,109.510,109.538,...,109.429,109.588,109.422,0.151,1,1,1,109.6031,109.5427,109.7239
62,63,USD_JPY,2020-01-16T10:00:00.000000000Z,6412,109.997,110.049,109.918,110.007,110.002,110.054,...,109.932,110.075,109.926,0.134,1,1,1,110.0884,110.0348,110.1956


In [23]:
df[df.SIGNAL==1].to_csv("./Data/Pairs/USD_JPY_H4_trades.csv")

In [24]:
plot_candles_mode(df_plot, "markers")

In [25]:
plot_candles_mode(df_plot, "lines")